<a href="https://colab.research.google.com/github/Charmiece/Final-Project---CreditCard-Fraud/blob/Afreensiddiqa/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://fraudcreditcard.s3.amazonaws.com/fraudTrain.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("fraudTrain.csv"), sep=",", header=True)
# Show DataFrame
df.show()

+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|_c0|trans_date_trans_time|             cc_num|            merchant|     category|   amt|      first|     last|gender|              street|                city|state|  zip|    lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  0|  2019-01-01 00:00:18|   2703186189

In [4]:
from pyspark import SparkFiles
url ="https://fraudcreditcard.s3.amazonaws.com/fraudTest.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("fraudTest.csv"), sep=",", header=True)
# Show DataFrame
df.show()

+---+---------------------+-----------+--------------------+--------------+------+---------+--------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+---------+-----------+--------+
|_c0|trans_date_trans_time|     cc_num|            merchant|      category|   amt|    first|    last|gender|              street|         city|state|  zip|    lat|     long|city_pop|                 job|       dob|           trans_num| unix_time|merch_lat| merch_long|is_fraud|
+---+---------------------+-----------+--------------------+--------------+------+---------+--------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+---------+-----------+--------+
|  0|      6/21/2020 12:14|2.29116E+15|fraud_Kirlin and ...| personal_care|  2.86|     Jeff| Elliott|     M|   351 Darlene Green|     Columbia|   SC|29209|33.9659| -8

In [41]:
# importing the packages
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import sklearn.exceptions
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import OneClassSVM
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
from imblearn.under_sampling import NearMiss 
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics, preprocessing, svm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report,accuracy_score
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
from imblearn.metrics import classification_report_imbalanced


from sklearn.ensemble import RandomForestClassifier

In [6]:
# loading data
data_train = pd.read_csv("https://fraudcreditcard.s3.amazonaws.com/fraudTrain.csv")
data_test = pd.read_csv("https://fraudcreditcard.s3.amazonaws.com/fraudTest.csv")

In [ ]:
data_train.info()

In [22]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  float64
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [8]:
print(data_train.shape,data_test.shape)

(1296675, 23) (555719, 23)


In [23]:
# taking smaller sample to run the model faster

df_train= data_train.sample(frac = 0.1,random_state=1)
df_test= data_test.sample(frac = 0.05,random_state=1)
print(df_train.shape,df_test.shape)
df_train.head(10)

(129668, 23) (27786, 23)


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
94102,94102,2019-02-25 08:24:40,374497717543058,fraud_Funk Group,grocery_net,20.00,Linda,Hurst,F,31701 Tucker Square Suite 893,...,47.1709,-100.7944,1190,"Designer, ceramics/pottery",1948-06-30,1595dec12f6f19ceaae9b7df0f8af5c0,1330158280,46.398331,-99.813959,0
198791,198791,2019-04-12 19:50:15,4428154703770710,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,Brittany,Guerra,F,79209 Gary Dale,...,46.5901,-117.1692,761,Chief Marketing Officer,1943-06-30,0ed26b649ed0fce94d8e632b7208dea0,1334260215,45.687331,-117.488135,0
1238587,1238587,2020-05-31 21:50:53,213148039875802,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,Jill,Jacobs,F,034 Kimberly Mountains,...,27.9551,-82.2966,79613,Environmental consultant,1978-11-30,7096316ec1a4b261e8613013827abae7,1370037053,27.254081,-81.974799,0
619078,619078,2019-09-20 17:06:52,2283743876903625,fraud_Conroy-Emard,food_dining,45.38,Shannon,Williams,F,9345 Spencer Junctions Suite 183,...,34.0770,-84.3033,165556,Prison officer,1997-12-27,d02c35d7bda19c3da84b6d01f02da7e8,1348160812,34.551957,-83.374265,0
573850,573850,2019-08-31 23:22:50,4798937978667559470,fraud_Adams-Barrows,health_fitness,27.52,Robert,Williams,M,09419 Lewis Rapids,...,43.6088,-83.9530,67858,Water quality scientist,1995-03-13,dccf059d9fdc15f1fb9d2abb50e7314b,1346455370,43.032957,-83.521294,0
1201872,1201872,2020-05-17 18:21:17,6538891242532018,fraud_Effertz LLC,entertainment,131.75,Jodi,Rodriguez,F,5796 Lee Coves Apt. 286,...,36.0244,-90.9288,7155,"Engineer, control and instrumentation",1977-03-23,4e700b0737fbfe948c3af0af594a964e,1368814877,35.068173,-90.274482,0
61984,61984,2019-02-05 21:47:54,4998109455173950,fraud_Morissette LLC,entertainment,146.69,Mary,Pruitt,F,2819 Luke Greens Suite 563,...,38.9621,-94.5959,545147,Counsellor,1987-11-18,dd7ea05d67a0ff509e3d0fb1fdfe7f38,1328478474,38.229734,-94.197224,0
552029,552029,2019-08-24 11:59:19,60422928733,fraud_Wiegand-Lowe,grocery_net,74.27,Jeffrey,Powers,M,38352 Parrish Road Apt. 652,...,33.6028,-81.9748,46944,Secondary school teacher,1942-04-02,214d540bdca14cdeb5eb347784d44f0c,1345809559,34.230048,-82.674878,0
1079425,1079425,2020-03-24 06:07:24,377264520876399,fraud_Harris Group,food_dining,11.22,Kara,Miles,F,2076 Thomas Roads Suite 970,...,34.3424,-80.5000,4424,Lawyer,1961-07-31,e7370494e1d3171f0ce1a5bdbae071f4,1364105244,33.700800,-80.342778,0
681734,681734,2019-10-18 03:41:04,4070511711385545,fraud_Dach-Nader,misc_net,3.96,Stephanie,Hopkins,F,44089 Carrie Crossing Suite 954,...,28.9814,-98.0156,2435,"Engineer, mining",1967-04-09,e2f08d1f890af7c6f4d3dfb6cc90a18a,1350531664,29.905143,-98.185725,0


In [24]:
df_train.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [25]:
data_test.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [26]:
# function to drop tbe columns
def dropCol(data):
    col_to_drop = ['trans_date_trans_time','Unnamed: 0','cc_num','first','last','trans_num']
    res = data.drop(col_to_drop,axis = 1)
    return res

In [27]:
# dropping the columns
# dropping the columns ['trans_date_trans_time','Unnamed: 0','cc_num','first','last','trans_num']
# train data set
df_train = dropCol(df_train)
# test data set
df_test = dropCol(df_test)

print ( df_train.shape, df_test.shape)

(129668, 17) (27786, 17)


In [28]:
# function to convert dob to years
def age_years(born):
    return 2022 - int(born[0:4])

# replacing the dob column with age column in our data set for test and train
df_train['age'] = df_train['dob'].apply(lambda x: age_years(x))



In [29]:
df_train

,merchant,category,amt,gender,street,city,state,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud,age
94102,fraud_Funk Group,grocery_net,20.00,F,31701 Tucker Square Suite 893,Wilton,ND,58579,47.1709,-100.7944,1190,"Designer, ceramics/pottery",1948-06-30,1330158280,46.398331,-99.813959,0,74
198791,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,F,79209 Gary Dale,Colton,WA,99113,46.5901,-117.1692,761,Chief Marketing Officer,1943-06-30,1334260215,45.687331,-117.488135,0,79
1238587,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,F,034 Kimberly Mountains,Brandon,FL,33510,27.9551,-82.2966,79613,Environmental consultant,1978-11-30,1370037053,27.254081,-81.974799,0,44
619078,fraud_Conroy-Emard,food_dining,45.38,F,9345 Spencer Junctions Suite 183,Alpharetta,GA,30009,34.0770,-84.3033,165556,Prison officer,1997-12-27,1348160812,34.551957,-83.374265,0,25
573850,fraud_Adams-Barrows,health_fitness,27.52,M,09419 Lewis Rapids,Bay City,MI,48706,43.6088,-83.9530,67858,Water quality scientist,1995-03-13,1346455370,43.032957,-83.521294,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944454,fraud_Reichert-Weissnat,travel,5.83,F,6911 Nicholas Keys Apt. 237,Armagh,PA,15920,40.4603,-79.0097,922,Early years teacher,1972-10-04,1358007774,40.522369,-79.314668,0,50
23683,"fraud_Baumbach, Strosin and Nicolas",shopping_pos,1.96,F,27479 Reeves Dale,Whaleyville,MD,21872,38.4121,-75.2811,718,Regulatory affairs officer,1985-03-19,1326572609,38.534192,-76.161654,0,37
296375,fraud_Lubowitz-Walter,kids_pets,26.59,F,72966 Shannon Pass Apt. 391,Bauxite,AR,72011,34.5091,-92.4828,4074,"Scientist, audiological",1975-12-20,1337963080,35.175647,-91.740062,0,47
306312,"fraud_Runolfsdottir, Mueller and Hand",entertainment,6.00,M,3283 James Station,Fayetteville,NC,28314,35.0583,-79.0080,238602,"Scientist, research (maths)",1993-05-14,1338216727,34.360264,-78.128199,0,29


In [30]:
df_train = df_train.drop(['dob'],axis =1)
df_train

,merchant,category,amt,gender,street,city,state,zip,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud,age
94102,fraud_Funk Group,grocery_net,20.00,F,31701 Tucker Square Suite 893,Wilton,ND,58579,47.1709,-100.7944,1190,"Designer, ceramics/pottery",1330158280,46.398331,-99.813959,0,74
198791,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,F,79209 Gary Dale,Colton,WA,99113,46.5901,-117.1692,761,Chief Marketing Officer,1334260215,45.687331,-117.488135,0,79
1238587,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,F,034 Kimberly Mountains,Brandon,FL,33510,27.9551,-82.2966,79613,Environmental consultant,1370037053,27.254081,-81.974799,0,44
619078,fraud_Conroy-Emard,food_dining,45.38,F,9345 Spencer Junctions Suite 183,Alpharetta,GA,30009,34.0770,-84.3033,165556,Prison officer,1348160812,34.551957,-83.374265,0,25
573850,fraud_Adams-Barrows,health_fitness,27.52,M,09419 Lewis Rapids,Bay City,MI,48706,43.6088,-83.9530,67858,Water quality scientist,1346455370,43.032957,-83.521294,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944454,fraud_Reichert-Weissnat,travel,5.83,F,6911 Nicholas Keys Apt. 237,Armagh,PA,15920,40.4603,-79.0097,922,Early years teacher,1358007774,40.522369,-79.314668,0,50
23683,"fraud_Baumbach, Strosin and Nicolas",shopping_pos,1.96,F,27479 Reeves Dale,Whaleyville,MD,21872,38.4121,-75.2811,718,Regulatory affairs officer,1326572609,38.534192,-76.161654,0,37
296375,fraud_Lubowitz-Walter,kids_pets,26.59,F,72966 Shannon Pass Apt. 391,Bauxite,AR,72011,34.5091,-92.4828,4074,"Scientist, audiological",1337963080,35.175647,-91.740062,0,47
306312,"fraud_Runolfsdottir, Mueller and Hand",entertainment,6.00,M,3283 James Station,Fayetteville,NC,28314,35.0583,-79.0080,238602,"Scientist, research (maths)",1338216727,34.360264,-78.128199,0,29


In [32]:
# Create our features
X = pd.get_dummies(df_train, columns=['merchant', 'category', 'gender', 'street', 
                'city', 'state', 'job'])

# Create our target
y = df_train['is_fraud']

In [33]:
X.describe()

,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,age,...,job_Video editor,job_Visual merchandiser,job_Volunteer coordinator,job_Warden/ranger,job_Waste management officer,job_Water engineer,job_Water quality scientist,job_Web designer,job_Wellsite geologist,job_Writer
count,129668.000000,129668.000000,129668.000000,129668.000000,1.296680e+05,1.296680e+05,129668.000000,129668.000000,129668.000000,129668.000000,...,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000
mean,70.061919,48769.624564,38.531795,-90.195962,8.856453e+04,1.349218e+09,38.532677,-90.195662,0.005437,48.721990,...,0.002560,0.000794,0.001697,0.002537,0.001967,0.004866,0.000370,0.002059,0.002159,0.000386
std,153.627239,26914.806946,5.062368,13.747697,3.015705e+05,1.279134e+07,5.094736,13.758655,0.073535,17.374394,...,0.050536,0.028173,0.041155,0.050307,0.044302,0.069589,0.019236,0.045331,0.046419,0.019633
min,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.325376e+09,19.035472,-166.657834,0.000000,17.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.630000,26041.000000,34.668900,-96.798000,7.410000e+02,1.338757e+09,34.751272,-96.899931,0.000000,35.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47.360000,48174.000000,39.346500,-87.458100,2.456000e+03,1.349244e+09,39.360173,-87.408320,0.000000,47.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,82.940000,72042.000000,41.846700,-80.158000,2.032800e+04,1.359235e+09,41.938529,-80.190236,0.000000,60.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,17897.240000,99783.000000,65.689900,-67.950300,2.906700e+06,1.371817e+09,66.645176,-66.963975,1.000000,98.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
# Import dependencies
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import EasyEnsembleClassifier

In [35]:
# Resample the training data with the BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X,y)
print(rf_model)

BalancedRandomForestClassifier(random_state=1)


In [37]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X)
balanced_accuracy_score(y, y_pred)

0.999740235571443

In [38]:
# Display the confusion matrix
# Calculating the confusion matrix.
y_pred = rf_model.predict(X)
cm = confusion_matrix(y, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"],
    columns=["Predicted high_risk", "Predicted low_risk"]
)

# Displaying results
display(cm_df)

,Predicted high_risk,Predicted low_risk
Actual high_risk,128896,67
Actual low_risk,0,705


In [42]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00    128963
          1       0.91      1.00      1.00      0.95      1.00      1.00       705

avg / total       1.00      1.00      1.00      1.00      1.00      1.00    129668



In [43]:
# List the features sorted in descending order by feature importance
feature_importances = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
feature_importances

[(0.36632722096706205, 'is_fraud'),
 (0.17849727494913012, 'amt'),
 (0.016520400651017123, 'unix_time'),
 (0.015287875888139903, 'category_shopping_net'),
 (0.014540104738150414, 'merch_lat'),
 (0.014145734039393986, 'merch_long'),
 (0.012055550690079391, 'lat'),
 (0.011571565941805083, 'long'),
 (0.011396611493750645, 'age'),
 (0.01075946242636187, 'city_pop'),
 (0.009442756548816761, 'zip'),
 (0.009299556040330094, 'category_misc_net'),
 (0.0065404388288600734, 'category_grocery_pos'),
 (0.006177936461648049, 'category_health_fitness'),
 (0.005074823637721482, 'category_gas_transport'),
 (0.004247367515965853, 'category_personal_care'),
 (0.004094068251374308, 'category_kids_pets'),
 (0.003854833638695664, 'category_home'),
 (0.0034935207265959257, 'category_food_dining'),
 (0.0030652932660021233, 'category_misc_pos'),
 (0.002895395540637625, 'gender_F'),
 (0.0022949466430028016, 'gender_M'),
 (0.0022248819543940444, 'category_travel'),
 (0.0020004303713323087, 'category_grocery_net'